# Handle my dataset questions crawled

In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('/kaggle/input/law-qa/lawlaboratory.questions.json')
df.shape

(363867, 9)

In [3]:
df.info()
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363867 entries, 0 to 363866
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   _id          363867 non-null  object
 1   title        363867 non-null  object
 2   description  246641 non-null  object
 3   date_answer  363867 non-null  object
 4   field        363867 non-null  object
 5   source_url   363867 non-null  object
 6   conclusion   363867 non-null  object
 7   reference    117226 non-null  object
 8   quote        117226 non-null  object
dtypes: object(9)
memory usage: 25.0+ MB


,_id,title,description,date_answer,field,source_url,conclusion,reference,quote
207531,{'$oid': '66635737c2f544363eee6bdb'},Bệnh viện không xử lý chất thải y tế theo quy ...,Hình thức xử lý bệnh viện không xử lý chất thả...,2018-01-09,Vi phạm hành chính,https://thuvienphapluat.vn/hoi-dap-phap-luat/3...,"[Ngày 14/11/2013, Chính phủ ban hành Nghị định...",NaN,NaN
218774,{'$oid': '66635746c2f544363eeebe1e'},Mẫu thẻ BHYT cũ đã cấp có được sử dụng hay không?,Mình có biết là theo Quyết định 1666 thì thẻ B...,2021-01-09,Bảo hiểm,https://thuvienphapluat.vn/hoi-dap-phap-luat/5...,[Theo Khoản 2 Điều 4 Quyết định 1666/QĐ-BHXH n...,NaN,NaN
277777,{'$oid': '666357a2c2f544363ef069d7'},"In Vietnam, how is the application for exempti...",NaN,2022-05-28,Thuế - Phí - Lệ phí,https://thuvienphapluat.vn/hoi-dap-phap-luat/5...,"[Pursuant to Clause 2 of the above Article, th...","Pursuant to Clause 2 of the above Article, the...",{'content': []}
175302,{'$oid': '66635717c2f544363eedb933'},Hợp đồng lao động dài hạn trong chỉ tiêu biên ...,Xin chào luật sư tư vấn. Tôi tên là Nga làm vi...,2016-09-05,Lao động - Tiền lương,https://thuvienphapluat.vn/hoi-dap-phap-luat/1...,"[Chào bạn., Trong thời gian chờ xin chỉ tiêu b...",NaN,NaN
12225,{'$oid': '6663561ac2f544363eea8cc1'},Đang có vợ hoặc đang có chồng mà chung sống vớ...,Đang có vợ hoặc đang có chồng mà chung sống vớ...,2017-02-10,Quyền dân sự,https://thuvienphapluat.vn/hoi-dap-phap-luat/2...,[Theo quy định tại Điều 51 Luật hôn nhân và gi...,NaN,NaN


In [4]:
df = df.dropna(subset=['title'])
df = df[df['title'] != '']
df.shape

(363866, 9)

In [5]:
viet_characters = "ÀÁẢÃẠĂẰẮẲẴẶÂẦẤẨẪẬàáảãạăằắẳẵặâầấẩẫậĐđÈÉẺẼẸÊỀẾỂỄỆèéẻẽẹêềếểễệÌÍỈĨỊìíỉĩịÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢòóỏõọôồốổỗộơờớởỡợÙÚỦŨỤƯỪỨỬỮỰùúủũụưừứửữựỲÝỶỸỴỳýỷỹỵ"

def contains_vietnamese_characters(title, vietnamese_chars):
    return any(char in vietnamese_chars for char in title)

df = df[df['title'].apply(lambda x: contains_vietnamese_characters(x, viet_characters))]

In [6]:
def process_quote(quote):
    if pd.isna(quote): 
        return np.nan 
    if 'content' in quote and not quote['content']:
        return np.nan
    return quote

df['quote'] = df['quote'].apply(process_quote)

In [7]:
def dropEmptyAnswer(row):
    if isinstance(row['conclusion'], list) and not row['conclusion'] and pd.isna(row['quote']):
        return True
    return False

df = df[~df.apply(dropEmptyAnswer, axis=1)]

df = df.reset_index(drop=True)
df.shape

(358322, 9)

In [8]:
def clean_conclusionArray(conclusion):
    words_to_remove = ["tải về", "mẫu đơn", "hình từ internet", "hình ảnh internet", "hình internet", "trân trọng"]
    if isinstance(conclusion, list):
        cleaned_conclusion1 = [word for word in conclusion if isinstance(word, str) and not any(remove_word in word.lower() for remove_word in words_to_remove)]
        cleaned_conclusion2 = [x for x in cleaned_conclusion1 if x] 
        return cleaned_conclusion2
    return conclusion

df['conclusion'] = df['conclusion'].apply(clean_conclusionArray)

In [9]:
def clean_title(title):
    for i, char in enumerate(title):
        if char.isupper():
            return title[i:]
    return title

df['title'] = df['title'].apply(clean_title)

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')

def count_tokens(text):
    tokens = tokenizer.tokenize(text)
    return len(tokens)

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [11]:
df['token_count'] = df['title'].apply(count_tokens)
df = df[df['token_count'] > 3].drop(columns=['token_count'])

In [12]:
df['date_answer'] = pd.to_datetime(df['date_answer'])

df.sort_values(by='date_answer', ascending=False, inplace=True)
df.drop_duplicates(subset='title', keep='first', inplace=True)

df['date_answer'] = df['date_answer'].dt.strftime('%Y-%m-%d')

In [13]:
df.shape

(357726, 9)

In [14]:
df.to_json('/kaggle/working/lawlaboratory.questions_cleaned.json', orient='records', lines=True, force_ascii=False)

In [15]:
df.iloc[100]

_id                         {'$oid': '667aa98e1766e264884710ce'}
title          Học viện Chính trị Quốc gia Hồ Chí Minh có bao...
description                                                  NaN
date_answer                                           2024-06-25
field                                          Bộ máy hành chính
source_url     https://thuvienphapluat.vn/hoi-dap-phap-luat/8...
conclusion                                                    []
reference      Tại Điều 4 Nghị định 48/2014/NĐ-CP quy định nh...
quote          {'name': 'Điều 4. Lãnh đạo Học viện', 'content...
Name: 355824, dtype: object

In [16]:
def merge_quote(quote):
    if isinstance(quote, dict):
        name = quote.get('name', ' ')
        content = ' '.join(quote.get('content', []))

        if name is None:
            name = ' '
        if content is None:
            content = ' '

        return f"{name}. {content}"
    else:
        return None


df['conclusion'] = df.apply(lambda row: merge_quote(row['quote']) if row['conclusion'] == [] else row['conclusion'], axis=1)

In [17]:
def is_invalid_conclusion(conclusion):
    if conclusion is None:
        return True
    if isinstance(conclusion, float) and np.isnan(conclusion):
        return True
    if isinstance(conclusion, list) and len(conclusion) == 0:
        return True
    return False

df = df[~df['conclusion'].apply(is_invalid_conclusion)]

In [18]:
def create_positive(row):
    conclusion_value = row['conclusion']

    if isinstance(conclusion_value, list):
        conclusion_value = ' '.join(conclusion_value)
    elif pd.isna(conclusion_value):
        conclusion_value = ''
    
    quote_value= ''
    if not pd.isna(row['quote']):
        reference = row['reference'] if not pd.isna(row['reference']) else ''
        merged_quote = merge_quote(row['quote'])
        quote_value = f"{reference} {merged_quote}"
        
    conclusion_tokens = count_tokens(conclusion_value)
    quote_tokens = count_tokens(quote_value)
    
    if conclusion_tokens >= quote_tokens:
        return conclusion_value
    else:
        return quote_value
    
df['positive'] = df.apply(create_positive, axis=1)

In [19]:
def printConclustion(df, target_title):
    conclusion_value = df.loc[df['title'] == target_title, 'conclusion'].values[0]

    if isinstance(conclusion_value, list):
        conclusion_value = '\n'.join(conclusion_value)

    if conclusion_value and isinstance(conclusion_value, str):
        print(f"'{target_title}': {conclusion_value}")
    else:
        print(f"'{target_title}'")

printConclustion(df, "Điều kiện tuyên bố mất tích là gì?")

'Điều kiện tuyên bố mất tích là gì?': Như vậy, điều kiện tuyên bố mất tích là:
- Biệt tích 02 năm liền trở lên;
- Đã áp dụng đầy đủ các biện pháp thông báo, tìm kiếm theo quy định của pháp luật về tố tụng dân sự nhưng vẫn không có tin tức xác thực về việc người đó còn sống hay đã chết.
- Có gửi đơn yêu cầu tuyên bố người mất tích đến Toà án.


In [20]:
df = df.drop(['date_answer', 'field', 'description', 'reference', 'quote', 'source_url'], axis=1)
df.sample(5)

,_id,title,conclusion,positive
48519,{'$oid': '66635646c2f544363eeb27f2'},Thủ tục kết nạp hội viên Hội Bảo vệ quyền tác ...,[Theo Điều 11 Điều lệ Hội Bảo vệ quyền tác phẩ...,Theo Điều 11 Điều lệ Hội Bảo vệ quyền tác phẩm...
265586,{'$oid': '66635788c2f544363ef0386d'},Hồ sơ đề nghị gia hạn giấy phép xây dựng bao g...,"[Như vậy, hồ sơ đề nghị gia hạn giấy phép xây ...",Tại Điều 51 Nghị định 15/2021/NĐ-CP có quy địn...
70868,{'$oid': '66635673c2f544363eeb880e'},Khách nước ngoài không thanh toán tiền phải là...,[Như tình hình bạn trình bày thì giữa công ty ...,Như tình hình bạn trình bày thì giữa công ty b...
340393,{'$oid': '6663583ac2f544363ef24c3c'},Quan hệ công tác của lực lượng tham gia bảo vệ...,"[Như vậy, quan hệ công tác của lực lượng tham ...",Tại Điều 5 Luật Lực lượng tham gia bảo vệ an n...
47971,{'$oid': '66635646c2f544363eeb24ec'},Không cho sinh viên nữ tham dự nghiên cứu khoa...,"[Như vậy, người nào có hành vi cản trở không c...",Căn cứ quy định khoản 1 Điều 10 Nghị định 125/...


In [21]:
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356686 entries, 0 to 356685
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   _id         356686 non-null  object
 1   title       356686 non-null  object
 2   conclusion  356686 non-null  object
 3   positive    356686 non-null  object
dtypes: object(4)
memory usage: 10.9+ MB


In [22]:
import json
with open('/kaggle/input/law-qa/query_set_evaluate.json', 'r') as file:
    test_queries = [json.loads(line) for line in file]

In [23]:
test_query_ids = [item['query_id'] for item in test_queries]

In [24]:
print(len(test_query_ids))

18591


In [25]:
df['_id_str'] = df['_id'].apply(lambda x: x['$oid'])

In [26]:
df_train = df[~df['_id_str'].isin(test_query_ids)]
df_train = df_train.drop(columns=['_id_str'])

In [27]:
df_train.reset_index(drop=True, inplace=True)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338113 entries, 0 to 338112
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   _id         338113 non-null  object
 1   title       338113 non-null  object
 2   conclusion  338113 non-null  object
 3   positive    338113 non-null  object
dtypes: object(4)
memory usage: 10.3+ MB


In [28]:
mydata_query_train = pd.DataFrame()

mydata_query_train['query'] = df_train['title']
mydata_query_train['positive'] = df_train['positive']

In [29]:
mydata_query_train = mydata_query_train.sample(frac=1).reset_index(drop=True)
mydata_query_train.sample(5)

,query,positive
336221,Thông báo nộp tiền tạm ứng chi phí định giá tà...,Thông báo nộp tiền tạm ứng chi phí định giá tà...
112099,Mua phải đồ ăn trộm thì xử lý như thế nào?,1/ Nếu việc bạn mua chiếc xe kia hoàn toàn khô...
138946,Xe gắn máy quay đầu xe tại nơi cấm quay đầu xe...,Theo quy định tại điểm h khoản 2 Điều 6 Nghị đ...
164268,Hồ sơ bổ sung nghiệp vụ kinh doanh chứng khoán...,Theo quy định tại Khoản 2 Điều 20 Thông tư 210...
154093,"Lái xe khách hết niên hạn sử dụng, bị xử phạt ...",Căn cứ quy định tại khoản 2 điều 4 nghị định 9...


In [30]:
mydata_query_train.to_json('/kaggle/working/mydata_trainset.json', orient='records')

# Merge ZaloAI dataset

In [31]:
import json
corpus = json.load(open('/kaggle/input/zaloai2021-legal-text-retrieval/legal_corpus.json'))

In [32]:
idx_mapping = {}
flattern_corpus =[]
for cc in corpus:
    law_id = cc['law_id']
    for c in cc['articles']:
        article_id = c['article_id']
        idx_mapping[f'{law_id}_{article_id}']=len(flattern_corpus)
        flattern_corpus.append({'doc_id':law_id + "_" +article_id,
                                'title': c['title'],
                               'text': c['text']})

In [33]:
flattern_corpus[1]

{'doc_id': '01/2009/tt-bnn_2',
 'title': 'Điều 2. Tổ chức lực lượng',
 'text': '1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân cấp xã nơi có đê phải tổ chức lực lượng lao động tại địa phương để tuần tra, canh gác đê và thường trực trên các điếm canh đê hoặc nhà dân khu vực gần đê (đối với những khu vực chưa có điếm canh đê), khi có báo động lũ từ cấp I trở lên đối với tuyến sông có đê (sau đây gọi tắt là lực lượng tuần tra, canh gác đê).\n2. Lực lượng tuần tra, canh gác đê được tổ chức thành các đội, do Ủy ban nhân dân cấp xã ra quyết định thành lập; từ 01 đến 02 kilômét đê thành lập 01 đội; mỗi đội có từ 12 đến 18 người, trong đó có 01 đội trưởng và 01 hoặc 02 đội phó. Danh sách thành viên đội tuần tra, canh gác đê được niêm yết tại điếm canh đê thuộc địa bàn được phân công.\n3. Khi lũ, bão có diễn biến phức tạp, kéo dài ngày, Uỷ ban nhân dân cấp xã có thể quyết định việc bổ sung thêm thành viên cho đội tuần tra, canh gác đê.'}

In [34]:
train_question = json.load(open('/kaggle/input/zaloai2021-legal-text-retrieval/train_question_answer.json'))

In [35]:
query_set = []

for query in train_question['items']:
    query_embedding = query['question']
    relevant_docs = []
    for doc in query['relevant_articles']:
        relevant_docs.append(doc['law_id'] + "_" + doc['article_id'])
    query_set.append({'query_id':query['question_id'],
                              'query' : query_embedding,
                              'relevant_docs' : relevant_docs
                            })

In [36]:
query_set[1]

{'query_id': 'ade2b2ee4f5b869f75f0d183902382af',
 'query': 'Phải thực hiện thao tác nạp mẫu vào bình chứa và xử lý mẫu sơ bộ bằng hóa chất như thế nào?',
 'relevant_docs': ['41/2020/tt-bca_11']}

In [37]:
import pickle

with open('/kaggle/working/idx_mapping', 'wb') as f:
    pickle.dump(idx_mapping, f)
with open('/kaggle/working/flattern_corpus', 'wb') as f:
    pickle.dump(flattern_corpus, f)
with open('/kaggle/working/query_set', 'wb') as f:
    pickle.dump(query_set, f)

In [38]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(query_set, test_size=0.2, random_state=42)

In [39]:
query_flatten = []
for query in train_set:
    question_id = query['query_id']
    question = query['query']
    for doc in query['relevant_docs']:
        query_flatten.append({
            'question_id':question_id,
            'question':question,
            'relevant_article_id': doc
        })

In [40]:
query_flatten[0]

{'question_id': 'a99b67ded3e927d0ca0dc43715f7ac72',
 'question': 'Công bố bản án, quyết định tòa án đối với doanh nghiệp nhỏ và vừa trên cổng thông tin điện tử được quy định như thế nào?',
 'relevant_article_id': '55/2019/nđ-cp_7'}

In [41]:
def find_pos(doc_id, corpus):
    pos = ""
    for doc in corpus:
        if (doc_id == doc['doc_id']):
            pos = doc['text']
            break
    return pos

In [42]:
zalo_query = []
for query in query_flatten:
    zalo_query.append({
        'query' : query['question'],
        'positive' : find_pos(query['relevant_article_id'], flattern_corpus)
    })

In [43]:
zalo_query[0]

{'query': 'Công bố bản án, quyết định tòa án đối với doanh nghiệp nhỏ và vừa trên cổng thông tin điện tử được quy định như thế nào?',
 'positive': '1. Việc công bố bản án, quyết định có hiệu lực pháp luật của tòa án trên cổng thông tin điện tử của tòa án được thực hiện theo Nghị quyết số 03/2017/NQ-HĐTP ngày 16 tháng 3 năm 2017 của Hội đồng Thẩm phán Tòa án nhân dân tối cao về việc công bố bản án, quyết định trên cổng thông tin điện tử của Tòa án hoặc văn bản quy phạm pháp luật sửa đổi, bổ sung hoặc thay thế Nghị quyết này.\n2. Việc công bố phán quyết, quyết định của trọng tài thương mại được thực hiện theo pháp luật trọng tài thương mại, thỏa thuận của các bên có liên quan đến phán quyết, quyết định đó.\n3. Việc công bố quyết định xử lý vụ việc cạnh tranh được thực hiện theo quy định của Luật Cạnh tranh và văn bản quy phạm pháp luật quy định chi tiết Luật này.\n4. Việc công bố quyết định xử lý vi phạm hành chính được thực hiện theo quy định của Luật Xử lý vi phạm hành chính và các văn

In [44]:
import pickle

file_path = '/kaggle/working/zalo_query_train'

with open(file_path, 'wb') as f:
    pickle.dump(zalo_query, f)

file_path = '/kaggle/working/zalo_test'

with open(file_path, 'wb') as f:
    pickle.dump(test_set, f)

In [45]:
zalo_query_df = pd.DataFrame(zalo_query, columns=['query', 'positive'])

In [46]:
mydata_80zalo_dataset_merged = pd.concat([mydata_query_train, zalo_query_df], ignore_index=True)

In [47]:
mydata_80zalo_dataset_merged.shape

(340746, 2)

In [48]:
# mydata_80zalo_dataset_merged.to_json('/kaggle/working/mydata_80zalo_dataset_merged.json', orient='records')